In [283]:
import csv, sys
from bs4 import BeautifulSoup
from datetime import datetime
from joblib import Parallel, delayed
import re
import pandas as pd

train_dataset_dir = './train/'
test_dataset_dir = './test/'

from os import listdir
from os.path import isfile, isdir, join


以下是loading data

In [303]:
train_df = None
test_df = None
train_files = listdir(train_dataset_dir)
test_files = listdir(test_dataset_dir)

train_df = pd.read_csv(train_dataset_dir + 'title_len.csv')
test_df = pd.read_csv(test_dataset_dir + 'title_len.csv')

In [304]:
for f in train_files:
        s = f[:-4]
        temp_df = pd.read_csv(train_dataset_dir + s + '.csv', lineterminator='\n')
        train_df[s] = temp_df[s]

In [306]:
for f in test_files:
        s = f[:-4]
        temp_df = pd.read_csv(test_dataset_dir + s + '.csv', lineterminator='\n')
        test_df[s] = temp_df[s]

In [307]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from matplotlib import pyplot as plt
import seaborn as sns

In [308]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.datasets import load_iris
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


解釋一下下面兩個transformer，可以當成standardScalar的感覺，call fit會把丟入的trainset去算dictionary跟mean(就跟SC的fit一樣，要先看train的平均一樣)，Call transform會把input transform成有popularity的datafram，所以可以放進pipeline

In [312]:
class AuthorTransFormer(BaseEstimator,TransformerMixin):
    def __init__(self, gate = 5):
        self.author_pop_dict = {}
        self.author_pop_mean = 0
        self.gate = 5
        
    def get_author_pop(self, author_arr):
        total_mean = []
        for i in author_arr:
            if i in self.author_pop_dict.keys():
                total_mean.append(self.author_pop_dict[i])
            else:
                total_mean.append(self.author_pop_mean)
        return total_mean

    def get_train_author_pop(self, author_arr, label_arr):
        popularity = {}
        for author, label in zip(author_arr, label_arr):
            if author in popularity.keys():
                popularity[author].append(int(label))
            else:
                popularity[author] = [int(label)]

        delete_keys = []
        for i in popularity.keys():
            if(len(popularity[i]) < self.gate):
                delete_keys.append(i)
            else:
                popularity[i] = sum(popularity[i]) / len(popularity[i])
                
        for j in delete_keys:
            popularity.pop(j, None)
        total_mean = []
        cnt = 0
        for i in author_arr:
            if i in popularity.keys():
                total_mean.append(popularity[i])
            else:
                cnt = cnt + 1
                total_mean.append(-1)

        mean = (sum(total_mean) + cnt)/ (len(total_mean) - cnt)
        return popularity, mean


    def fit(self,X,y=None):
        self.author_pop_dict, self.author_pop_mean = self.get_train_author_pop(X['author'], X['label'])
        return self
    
    def transform(self,X,y=None):
        X['popularity'] = self.get_author_pop(X['author'])
        X= X.drop(['author'], axis=1)
        return X

In [411]:
class TopicsTransFormer(BaseEstimator,TransformerMixin):
    def __init__(self, gate = 5):
        self.topics_pop_dict = {}
        self.topics_pop_mean = 0
        self.gate = 5
        
    def get_topics_pop(self,topics_arr):
        total_mean = []
        for topics in topics_arr :
            tps_val = -1
            choose_tps = ''
            for tps in topics.split():
                if tps in self.topics_pop_dict.keys():
                    if tps_val < self.topics_pop_dict[tps]:
                        tps_val = self.topics_pop_dict[tps]
                        choose_tps = tps
            if topics == '' or choose_tps == '':
                total_mean.append(self.topics_pop_mean)
            else:
                total_mean.append(self.topics_pop_dict[choose_tps])
        return total_mean

    def get_train_topics_pop(self, topics_arr, label_arr):
        popularity = {}
        for topics, label in zip(topics_arr, label_arr):
            for tps in topics.split():
                if tps in popularity.keys():
                    popularity[tps].append(int(label))
                else:
                    popularity[tps] = [int(label)]

        delete_keys = []
        for i in popularity.keys():
            if len(popularity[i]) < self.gate:
                delete_keys.append(i)
            else:
                popularity[i] = sum(popularity[i]) / len(popularity[i])
        for j in delete_keys:
            popularity.pop(j, None)

        total_mean = []
        cnt_non_tps = 0
        for topics in topics_arr:
            tps_val = -1
            choose_tps = ''
            for tps in topics.split():
                if tps not in popularity.keys():
                    continue
                if tps_val < popularity[tps]:
                    tps_val = popularity[tps]
                    choose_tps = tps
            if topics == '' or choose_tps == '':
                cnt_non_tps += 1
                total_mean.append(-1)
            else:
                total_mean.append(popularity[choose_tps])

        mean = (sum(total_mean) + cnt_non_tps) / (len(total_mean) - cnt_non_tps)
        return popularity, mean


    def fit(self,X,y=None):
        self.topics_pop_dict, self.topics_pop_mean = self.get_train_topics_pop(X['topics'].fillna(""), X['label'])
        return self
    
    def transform(self,X,y=None):
        X['topics_pop'] = self.get_topics_pop(X['topics'].fillna(""))
        if 'label' in X.columns:
            X= X.drop(['topics', 'label'], axis=1)
        else:
            X= X.drop(['topics'], axis=1)
        return X
    

In [412]:
class ChannelTransFormer(BaseEstimator,TransformerMixin):
    def __init__(self, gate = 5):
        self.author_pop_dict = {}
        self.author_pop_mean = 0
        self.gate = 5
        
    def get_author_pop(self, author_arr):
        total_mean = []
        for i in author_arr:
            if i in self.author_pop_dict.keys():
                total_mean.append(self.author_pop_dict[i])
            else:
                total_mean.append(self.author_pop_mean)
        return total_mean

    def get_train_author_pop(self, author_arr, label_arr):
        popularity = {}
        for author, label in zip(author_arr, label_arr):
            if author in popularity.keys():
                popularity[author].append(int(label))
            else:
                popularity[author] = [int(label)]

        delete_keys = []
        for i in popularity.keys():
            if(len(popularity[i]) < self.gate):
                delete_keys.append(i)
            else:
                popularity[i] = sum(popularity[i]) / len(popularity[i])
                
        for j in delete_keys:
            popularity.pop(j, None)
        total_mean = []
        cnt = 0
        for i in author_arr:
            if i in popularity.keys():
                total_mean.append(popularity[i])
            else:
                cnt = cnt + 1
                total_mean.append(-1)

        mean = (sum(total_mean) + cnt)/ (len(total_mean) - cnt)
        return popularity, mean


    def fit(self,X,y=None):
        self.author_pop_dict, self.author_pop_mean = self.get_train_author_pop(X['channel'], X['label'])
        return self
    
    def transform(self,X,y=None):
        X['channel_pop'] = self.get_author_pop(X['channel'])
        X= X.drop(['channel'], axis=1)
        return X

注意，authortrans一定要擺在topic的前面，topics一定要擺在SC的前面(因為中間輸出會drop 有文字的colums)

In [413]:
from xgboost import XGBClassifier
from sklearn.decomposition import PCA

pipe1 = Pipeline([('author', AuthorTransFormer()),
                  ('channel', ChannelTransFormer()),
                  ('topics', TopicsTransFormer()),
                  ('std', StandardScaler()),
#                   ('pca', PCA()),
                  ('clf', LogisticRegression(C=0.0001, random_state=17, n_jobs=4))])

注意! 選擇需要的feature，但是label、topics、author一定要放!(除非沒放topic跟authortransformer)

In [414]:
print(train_df.columns)

Index(['id', 'title_len', 'h2', 'dayhour', 'h1', 'author_post', 'channel',
       'author', 'label', 'topics', 'nn_percentage', 'title', 'cnt_iframe',
       'cnt_img', 'rb_percentage', 'topics_pop_5', 'jj_percentage', 'weekday',
       'links', 'topics_pop', 'topics_pop_10', 'topics_pop_25',
       'topics_pop_15', 'cnt_vdo', 'article', 'popularity', 'article_len',
       'topics_pop_20', 'vb_percentage'],
      dtype='object')


In [415]:
X_train = train_df[['dayhour', 'author_post', 'nn_percentage', 'cnt_iframe','cnt_img','rb_percentage', 'jj_percentage','links', 'weekday', 'popularity', 'cnt_vdo', 'article_len', 'vb_percentage', 'topics', 'author', 'label', 'channel']]

In [416]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, train_df['label'], test_size=0.3, random_state=1)

In [417]:
pipe1.fit(X_train, y_train)

Pipeline(steps=[('author', AuthorTransFormer()),
                ('channel', ChannelTransFormer()),
                ('topics', TopicsTransFormer()), ('std', StandardScaler()),
                ('clf',
                 LogisticRegression(C=0.0001, n_jobs=4, random_state=17))])

In [418]:
from sklearn.metrics import roc_auc_score
score = roc_auc_score(y_val, pipe1.predict_proba(X_val)[:,1])

In [419]:
print(score)

0.5452264226672794
